# Preparation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/MyDrive/Dibimbing/Data

/content/gdrive/MyDrive/Dibimbing/Data


In [3]:
# Packages
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Data
df = pd.read_excel('PRODUCTIVITY DESEMBER 2021.XLSX')

# Data Profiling

In [5]:
df.sample(5)

,Queue,Product,Product Short Description,Confirmed by,Confirmation Date,Confirmation Time
67153,INBOUND,970761.0,270 CUP X24 FRESTEA JASMINE,IDSUTRISNZZ3,2021-12-23,07:54:15
71037,OUTBOUND,970896.0,180 CBA X24 NUTRIBOOST COFFEE,IDSUDIARRO,2021-12-28,23:41:00
71184,OUTBOUND,970116.0,330 CAN X24 COCA-COLA ZERO,IDSUDIARRO,2021-12-29,01:23:54
4157,OUTBOUND,970927.0,300 PET X12 MMP ORANGE P4500,IDNURDIANAS,2021-12-02,23:35:23
51209,OUTBOUND,973979.0,1.5 PET X12 FANTA STRAWBERRY,IDROHIMAB3,2021-12-15,16:38:18


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76185 entries, 0 to 76184
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Queue                      76185 non-null  object        
 1   Product                    75936 non-null  float64       
 2   Product Short Description  75936 non-null  object        
 3   Confirmed by               76185 non-null  object        
 4   Confirmation Date          76185 non-null  datetime64[ns]
 5   Confirmation Time          76185 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 3.5+ MB


Data Description : 

1. Queue : Workload Classification
2. Product : Product ID number
3. Confirmed by : Warehouse order processor
4. Confirmation Date : Warehouse order confirmation date
5. Confirmation Time : Warehouse order confirmation time

Notes :
Pay attention to the quantity for every Product. The product have different quantity in which sum up in pallet quantity (you can assume that 1 pallet for each row).

In [7]:
df['Queue'].unique()

array(['OUTBOUND', 'OUT-CONT', 'O-PND-OUT', 'NARROW-RPL', 'NARROW-OUT',
       'NARROW', 'INTERNAL', 'INBOUND', 'INB-PROD'], dtype=object)

In [8]:
df['Queue'].value_counts()

OUTBOUND      21894
INBOUND       19986
INB-PROD      14467
OUT-CONT       9943
INTERNAL       3438
O-PND-OUT      2449
NARROW         2419
NARROW-OUT     1418
NARROW-RPL      171
Name: Queue, dtype: int64

# Handling Missing Value

In [9]:
#cek missing value -> Mendeteksi missing value menggunakan NA
missing_data = df.isnull().sum(axis=0).reset_index()
missing_data.columns = ['variable', 'missing values']
missing_data['filling factor (%)']=(df.shape[0]-missing_data['missing values'])/df.shape[0]*100
missing_data.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,Product,249,99.673164
1,Product Short Description,249,99.673164
2,Queue,0,100.000000
3,Confirmed by,0,100.000000
4,Confirmation Date,0,100.000000
5,Confirmation Time,0,100.000000


In [10]:
df = df.dropna()
df['Queue'].value_counts()

OUTBOUND      21894
INBOUND       19986
INB-PROD      14467
OUT-CONT       9943
INTERNAL       3437
NARROW         2419
O-PND-OUT      2201
NARROW-OUT     1418
NARROW-RPL      171
Name: Queue, dtype: int64

In [11]:
df['Product'] = df['Product'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75936 entries, 0 to 76184
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Queue                      75936 non-null  object        
 1   Product                    75936 non-null  int64         
 2   Product Short Description  75936 non-null  object        
 3   Confirmed by               75936 non-null  object        
 4   Confirmation Date          75936 non-null  datetime64[ns]
 5   Confirmation Time          75936 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 4.1+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Modeling Preparation

In [12]:
# create new dataframe for model
df_model = pd.DataFrame(columns = ['Date', 'Queue'])
df_model['Date'] = df['Confirmation Date']
df_model['Queue'] = df['Queue']
df_model

,Date,Queue
0,2021-12-01,OUTBOUND
1,2021-12-01,OUTBOUND
2,2021-12-01,OUTBOUND
3,2021-12-01,OUTBOUND
4,2021-12-01,OUTBOUND
...,...,...
76180,2021-12-30,INB-PROD
76181,2021-12-30,INB-PROD
76182,2021-12-30,INB-PROD
76183,2021-12-30,INB-PROD


In [13]:
# add Count for target variable
df_model = df_model.groupby(['Date', 'Queue']).size().reset_index(name="Count")
df_model['Count'] = df_model['Count'].astype(int)
df_model.tail(8)

,Date,Queue,Count
225,2021-12-30,OUTBOUND,339
226,2021-12-31,INBOUND,274
227,2021-12-31,INTERNAL,77
228,2021-12-31,NARROW-OUT,19
229,2021-12-31,NARROW-RPL,1
230,2021-12-31,O-PND-OUT,96
231,2021-12-31,OUT-CONT,13
232,2021-12-31,OUTBOUND,361


In [14]:
# Pivotting table
df_pivot = df_model.pivot( index='Date', columns='Queue' , values='Count' )
df_pivot

Queue,INB-PROD,INBOUND,INTERNAL,NARROW,NARROW-OUT,NARROW-RPL,O-PND-OUT,OUT-CONT,OUTBOUND
Date,,,,,,,,,
2021-12-01,2031.0,2970.0,531.0,550.0,171.0,10.0,149.0,1442.0,1955.0
2021-12-02,2186.0,3405.0,658.0,398.0,374.0,25.0,283.0,927.0,2268.0
2021-12-03,1644.0,3517.0,386.0,386.0,201.0,36.0,172.0,733.0,2040.0
2021-12-04,803.0,3195.0,303.0,249.0,342.0,31.0,225.0,815.0,2301.0
2021-12-05,NaN,1727.0,145.0,28.0,41.0,4.0,41.0,10.0,481.0
2021-12-06,29.0,73.0,14.0,7.0,NaN,1.0,12.0,112.0,256.0
2021-12-07,128.0,233.0,106.0,19.0,27.0,NaN,62.0,210.0,653.0
2021-12-08,245.0,176.0,64.0,21.0,15.0,3.0,39.0,60.0,202.0
2021-12-09,426.0,330.0,71.0,93.0,26.0,NaN,96.0,NaN,524.0


In [15]:
# drop NaN value for off day
df_pivot = df_pivot.drop(pd.to_datetime('2021-12-05'))
df_pivot

Queue,INB-PROD,INBOUND,INTERNAL,NARROW,NARROW-OUT,NARROW-RPL,O-PND-OUT,OUT-CONT,OUTBOUND
Date,,,,,,,,,
2021-12-01,2031.0,2970.0,531.0,550.0,171.0,10.0,149.0,1442.0,1955.0
2021-12-02,2186.0,3405.0,658.0,398.0,374.0,25.0,283.0,927.0,2268.0
2021-12-03,1644.0,3517.0,386.0,386.0,201.0,36.0,172.0,733.0,2040.0
2021-12-04,803.0,3195.0,303.0,249.0,342.0,31.0,225.0,815.0,2301.0
2021-12-06,29.0,73.0,14.0,7.0,NaN,1.0,12.0,112.0,256.0
2021-12-07,128.0,233.0,106.0,19.0,27.0,NaN,62.0,210.0,653.0
2021-12-08,245.0,176.0,64.0,21.0,15.0,3.0,39.0,60.0,202.0
2021-12-09,426.0,330.0,71.0,93.0,26.0,NaN,96.0,NaN,524.0
2021-12-10,430.0,194.0,57.0,64.0,7.0,2.0,14.0,77.0,675.0


In [16]:
# fill NaN value for working day
df_pivot = df_pivot.fillna(df_pivot.mean().round())
df_pivot

Queue,INB-PROD,INBOUND,INTERNAL,NARROW,NARROW-OUT,NARROW-RPL,O-PND-OUT,OUT-CONT,OUTBOUND
Date,,,,,,,,,
2021-12-01,2031.0,2970.0,531.0,550.0,171.0,10.0,149.0,1442.0,1955.0
2021-12-02,2186.0,3405.0,658.0,398.0,374.0,25.0,283.0,927.0,2268.0
2021-12-03,1644.0,3517.0,386.0,386.0,201.0,36.0,172.0,733.0,2040.0
2021-12-04,803.0,3195.0,303.0,249.0,342.0,31.0,225.0,815.0,2301.0
2021-12-06,29.0,73.0,14.0,7.0,55.0,1.0,12.0,112.0,256.0
2021-12-07,128.0,233.0,106.0,19.0,27.0,7.0,62.0,210.0,653.0
2021-12-08,245.0,176.0,64.0,21.0,15.0,3.0,39.0,60.0,202.0
2021-12-09,426.0,330.0,71.0,93.0,26.0,7.0,96.0,397.0,524.0
2021-12-10,430.0,194.0,57.0,64.0,7.0,2.0,14.0,77.0,675.0


# Data Modeling

In [17]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# contrived dataset
data_inbprod = df_pivot['INB-PROD'].values
data_inbound = df_pivot['INBOUND'].values
data_internal = df_pivot['INTERNAL'].values
data_narrow = df_pivot['NARROW'].values
data_narrowo = df_pivot['NARROW-OUT'].values
data_narrowr = df_pivot['NARROW-RPL'].values
data_pndo = df_pivot['O-PND-OUT'].values
data_outcont = df_pivot['OUT-CONT'].values
data_outbound = df_pivot['OUTBOUND'].values

In [18]:
# fit model
model_inbprod = ExponentialSmoothing(data_inbprod)
model_fit = model_inbprod.fit()
# make prediction
yhat_inbprod = model_fit.predict(len(data_inbprod), len(data_inbprod))
print(yhat_inbprod)

[579.]


In [19]:
# fit model
model_inbound = ExponentialSmoothing(data_inbound)
model_fit = model_inbound.fit()
# make prediction
yhat_inbound = model_fit.predict(len(data_inbound), len(data_inbound))
print(yhat_inbound)

[272.37537471]


In [20]:
# fit model
model_internal = ExponentialSmoothing(data_internal)
model_fit = model_internal.fit()
# make prediction
yhat_internal = model_fit.predict(len(data_internal), len(data_internal))
print(yhat_internal)

[77.56084803]


In [21]:
# fit model
model_narrow = ExponentialSmoothing(data_narrow)
model_fit = model_narrow.fit()
# make prediction
yhat_narrow = model_fit.predict(len(data_narrow), len(data_narrow))
print(yhat_narrow)

[100.]


In [22]:
# fit model
model_narrowo = ExponentialSmoothing(data_narrowo)
model_fit = model_narrowo.fit()
# make prediction
yhat_narrowo = model_fit.predict(len(data_narrowo), len(data_narrowo))
print(yhat_narrowo)

[16.49392179]


In [23]:
# fit model
model_narrowr = ExponentialSmoothing(data_narrowr)
model_fit = model_narrowr.fit()
# make prediction
yhat_narrowr = model_fit.predict(len(data_narrowr), len(data_narrowr))
print(yhat_narrowr)

[1.59033539]


In [24]:
# fit model
model_pndo = ExponentialSmoothing(data_pndo)
model_fit = model_pndo.fit()
# make prediction
yhat_pndo = model_fit.predict(len(data_pndo), len(data_pndo))
print(yhat_pndo)

[69.47726333]


In [25]:
# fit model
model_outcont = ExponentialSmoothing(data_outcont)
model_fit = model_outcont.fit()
# make prediction
yhat_outcont = model_fit.predict(len(data_outcont), len(data_outcont))
print(yhat_outcont)

[48.71085995]


In [26]:
# fit model
model_outbound = ExponentialSmoothing(data_outbound)
model_fit = model_outbound.fit()
# make prediction
yhat_outbound = model_fit.predict(len(data_outbound), len(data_outbound))
print(yhat_outbound)

[370.73368631]


In [27]:
yhat_fls = yhat_pndo + yhat_internal
yhat_rts = yhat_narrow + yhat_narrowo + yhat_narrowr
yhat_fld_inb = yhat_inbprod + yhat_inbound
yhat_fld_outb = yhat_outcont + yhat_outbound

In [28]:
print('Next day prediction for Forklift Inbound workload is', yhat_fld_inb, 'and so, the manpower need for this workload is', (yhat_fld_inb/175).round().astype(int))
print('Next day prediction for Forklift Outbound workload is', yhat_fld_outb, 'and so, the manpower need for this workload is', (yhat_fld_outb/175).round().astype(int))
print('Next day prediction for Forklift Internal+PND workload is', yhat_fls, 'and so, the manpower need for this workload is', (yhat_fls/175).round().astype(int))
print('Next day prediction for RTS workload is', yhat_rts, 'and so, the manpower need for this workload is', (yhat_rts/60).round().astype(int))

Next day prediction for Forklift Inbound workload is [851.37537471] and so, the manpower need for this workload is [5]
Next day prediction for Forklift Outbound workload is [419.44454626] and so, the manpower need for this workload is [2]
Next day prediction for Forklift Internal+PND workload is [147.03811136] and so, the manpower need for this workload is [1]
Next day prediction for RTS workload is [118.08425717] and so, the manpower need for this workload is [2]
